In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('train.csv')
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [5]:
###Drop Nan Values
df=df.dropna()


In [6]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [7]:
y=df['label']

In [9]:
X.shape,y.shape

((18285, 4), (18285,))

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [11]:
voc_size=5000

In [12]:
messages=X.copy()

In [13]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4934, 2977, 4007, 4882, 2011, 2285, 2960, 2897, 4999, 3167],
 [4554, 3769, 187, 6, 90, 854, 3797],
 [1036, 4207, 2799, 3095],
 [3230, 3598, 258, 915, 14, 100],
 [2551, 90, 1663, 2670, 4966, 1307, 90, 1035, 4033, 1497],
 [1800,
  2957,
  4494,
  904,
  4382,
  3347,
  4014,
  576,
  2070,
  4296,
  4625,
  564,
  1593,
  925,
  3797],
 [2302, 3150, 4116, 3370, 4308, 956, 1784, 666, 228, 4988, 2553],
 [2456, 3581, 3335, 4598, 3573, 4159, 3347, 4483, 228, 4988, 2553],
 [631, 4896, 1641, 620, 903, 4238, 856, 2104, 3347, 4827],
 [931, 2238, 853, 4176, 1358, 4047, 2766, 3519],
 [715, 199, 1709, 3648, 2079, 4644, 1266, 3657, 925, 864, 2732],
 [915, 4893, 2011, 4238, 3347, 3573],
 [1361, 2359, 1079, 718, 4558, 2437, 4146, 522, 4753],
 [1210, 3279, 4624, 2413, 2212, 4735, 2518, 228, 4988, 2553],
 [1999, 1566, 3454, 4965, 3738, 228, 4988, 2553],
 [446, 262, 99, 363, 3620, 4293, 649, 3878, 4839, 1650],
 [2067, 122, 3769],
 [3007, 4344, 638, 3313, 3347, 1667, 2613, 3797],
 [885, 1182, 187, 119, 

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2897 4999 3167]
 [   0    0    0 ...   90  854 3797]
 [   0    0    0 ... 4207 2799 3095]
 ...
 [   0    0    0 ...  228 4988 2553]
 [   0    0    0 ...   19 4682 4763]
 [   0    0    0 ... 4005 4515 1077]]


In [21]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4934,
       2977, 4007, 4882, 2011, 2285, 2960, 2897, 4999, 3167], dtype=int32)

In [22]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(18285, (18285,))

In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 15ms/step - loss: 0.4460 - accuracy: 0.7554 - val_loss: 0.2100 - val_accuracy: 0.9112
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1272 - accuracy: 0.9523 - val_loss: 0.2269 - val_accuracy: 0.9125
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0835 - accuracy: 0.9716 - val_loss: 0.2423 - val_accuracy: 0.9137
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0540 - accuracy: 0.9822 - val_loss: 0.3194 - val_accuracy: 0.9125
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0372 - accuracy: 0.9883 - val_loss: 0.3597 - val_accuracy: 0.9143
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0186 - accuracy: 0.9948 - val_loss: 0.4611 - val_accuracy: 0.9085
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0089 - accuracy: 0.9979 - val_loss: 0.5266 - val_accuracy: 0.910

In [27]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3093,  326],
       [ 240, 2376]])

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9062137531068766